In [1]:
import numpy as np
import cv2

In [2]:
import os
os.getcwd()

'c:\\cv\\opencv\\Scripts\\local_repo\\2017-2-CSP-MuJe-6\\OpenCV Tutorial'

감지 후 네모 그려주는 함수 - 여기서 padding 값은 실제 디텍션모다 ***크게*** 만들어줌

In [2]:
def draw_detections(img, rects, thickness = 1):
    
    for x, y, w, h in rects: # Classifier의 리턴 값 rect는 좌표 2개 형식임 -> [ (2, 3) , (3, 4) ]

        pad_w, pad_h = int(0.15*w), int(0.05*h) # 여기서 패딩 값 조정
        
        cv2.rectangle(img, (x+pad_w, y+pad_h), (x+w-pad_w, y+h-pad_h), (0, 255, 0), thickness)
        # 이미지는 상단 좌측에서부터 (0,0)에서 시작하므로 

여기가 (pre-training된) 캐스케이드 필터  디렉토리

In [3]:
cascade_path = "C:\\cv\\Haar Cascades models (XML)\\"

In [15]:
cap.release()

In [13]:
# ShiTomasi corner detection parameters
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Lucas Kanade optical flow parameters
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Random colors
color = np.random.randint(0,255,(100,3))
cap = cv2.VideoCapture(0)

In [14]:
# Cascade settings
face_cascade = cv2.CascadeClassifier(cascade_path + 'haarcascade_frontalface_default.xml')
upper_cascade = cv2.CascadeClassifier(cascade_path + 'Head & Shoulders.xml')

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = upper_cascade.detectMultiScale(old_gray, 1.3, 5)

mask = np.zeros_like(old_frame)


while cap.isOpened():
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY); 

     # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, gray, p0, None, **lk_params)
    
    
     # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)

    
    cv2.imshow('img', frame)
    
# "Q" to exit & release cap    
    ch = 0xFF & cv2.waitKey(1)
    if ch == 27:
        cap.release()
        break
        
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
cv2.destroyAllWindows()


error: C:\projects\opencv-python\opencv\modules\video\src\lkpyramid.cpp:1244: error: (-215) (npoints = prevPtsMat.checkVector(2, CV_32F, true)) >= 0 in function cv::`anonymous-namespace'::SparsePyrLKOpticalFlowImpl::calc


# Sparse Optical flow
- tracks pre-defined features

In [4]:
# ShiTomasi corner detection parameters
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Lucas Kanade optical flow parameters
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Random colors
color = np.random.randint(0,255,(100,3))
cap = cv2.VideoCapture(0)

NameError: name 'cv2' is not defined

In [ ]:
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
        
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
cv2.destroyAllWindows()
cap.release()

In [8]:
cap.release()

# Dense Optical flow
- attempts to track ALL pixels 

In [3]:
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255

while(1):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('opticalfb.png',frame2)
        cv2.imwrite('opticalhsv.png',bgr)
    prvs = next
cap.release()
cv2.destroyAllWindows()